# Solution to Homework 3 
## Name: $Supreeth Mohan$, $Yashika Patil$, $Sandeep Kasiraju$
 

## Problem 1                                                                                    

## Part A
### The C program given below will accept N as input and output the approximate value computed by the Monte Carlo method.

```c
#include <stdio.h>
#include <stdlib.h> 
#include <time.h> 
#include <math.h>
#include <omp.h>


double function(double x[], int dim);

double sampleInterval(double , double );

int main(int argc, char **argv)
{
  double startTime = omp_get_wtime();
  unsigned long long N = atoll(argv[1]);

  double a = -1.0;
  double b = 1.0;

  const int dimension=atoll(argv[2]);
  double x[dimension];
  double V=1024;
  double sum = 0.0;
  srand(time(NULL));
  unsigned long long i;

  for (i=0;i<N;++i)
  {
    for (int j=0;j<dimension;j++)
    {
      x[j] = sampleInterval(a,b);
    }

    double func_value  = function(x,dimension);
    
    sum = sum + func_value;
    
  }
  
  double integral =  (V/N) * sum;
  double endTime = omp_get_wtime();
  printf("%llu %f %f %f\n",N,integral,V,endTime-startTime);

  return 0;
}

double function(double x[], int dim) 
{
  double sum = 1.0;
  for (int i=0;i<dim;i++)
  {
    sum = sum + x[i];
  }
  return sum;
}


double sampleInterval(double a, double b) 
{
  double x = ((double)(rand()/(double)RAND_MAX)*(b-a))+a;
  return x;
}
```

### For this homework, we are passing dimension dynamically. Our program accepts 3 arguments, i.e., N, Dimension, and Number of Cores. 

## Plot of N vs Integral

### The integral converges to 1024 at the rate of N$^{(-1/2)}$

![Problem1_PartAImage.png](Problem1_PartAImage.png)

## Problem 2

## Part A and Part B

### Parallelized code using reduction and measured time using OMP_Timer and clock() in C  

```c
#include <stdio.h>
#include <stdlib.h> 
#include <time.h> 
#include <math.h>
#include <omp.h>

double function(double x[], int dim);

double sampleInterval(double , double );

int main(int argc, char **argv)
{
  unsigned int seed = (unsigned int)time(NULL);
  double startTime = omp_get_wtime();
  omp_set_num_threads(4);
  unsigned long long N = atoll(argv[1]);
  clock_t start_time = clock();
  double a = -1.0;
  double b = 1.0;
  int dimension=atoll(argv[2]);
  double x[dimension];
  double V=1024;
  unsigned long long i;
  double sum_thread[4] = {0};
  double func_value=0.0;
  double sum1 = 0.0;
  double integral=0.0;
  
  #pragma omp parallel for private(x) reduction(+:integral)
  for (i=0;i<N;++i)
  {
    double x[dimension];
    double sum = 1.0;
    for (int j=0;j<dimension;j++)
    {
      x[j]= ((double)(rand_r(&seed)/(double)RAND_MAX)*(b-a))+a;
      sum = sum + x[j]; 
    }

    integral +=  sum;
  }
    
  double integral1 = (V/N) * integral;  
  double time = ((double)clock() - start_time)/CLOCKS_PER_SEC;
  double endTime = omp_get_wtime();
  
  printf("%llu %f %f %f %f\n",N,integral1,V,time,endTime-startTime);
  
  return 0;
}


double function(double x[], int dim) 
{ 
  
  double sum = 1.0;
  for (int i=0;i<dim;i++)
  {
    sum = sum + x[i];
  }
  return sum;
 
 
}

double sampleInterval(double a, double b) 
{
  double x;
  x = ((double)(rand()/(double)RAND_MAX)*(b-a))+a;
  return x;
  
}
```

### We used 4 threads and used omp_get_max_threads() to print the number of threads, as seen in the image (check for last number in the terminal) below:
![Problem2_PartA_B_SC_Threads_htop.png](Problem2_PartA_B_SC_Threads_htop.png)
![Problem2_PartA_B_SC_Threads_terminal.png](Problem2_PartA_B_SC_Threads_terminal.png)

## Part B
### The comparison between clock time and omp_timer is shown in the image below. As expected, the clock() time is more than that of omp time.

![Problem2_PartBImage.png](Problem2_PartBImage.png)

## Problem 3
## Part A

###  OpenMP parallelized code implemeting Rosenbrock function. We used a void function, fun() ,which is a test function.

```c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <omp.h>
 
double function(double *x, int dim);
 
double variable(double , double, unsigned int *);
 
int main(int argc, char **argv)
{
    fun();
 
  return 0;
}

void fun()
{
    double x[10]={0};
    double val=function(x,10);
    printf("%f",val);
} 

double function(double *x, int dim)
{
  double f = 0.0;
  for(int i=0; i<dim-1; i++)
  {
    f += (-1*((pow(1-x[i],2) + 100*(pow(x[i+1]- pow(x[i],2),2)))));
  }
  return exp(f);
}
```

### Test Case: 
#### We passed x[10] = {0}, and computed the Rosenbrock function. The following image shows the output which is same as exp(-9) when calculated with the help of scientific calculator.

![Problem3_PartAImage.png](Problem3_PartAImage.png)

## Part B

### We are using reduction and critical parallelization. We used four Anvil job scripts, namely Strong scaling reduction, Strong scaling critical, Weak scaling reduction, Weak scaling critical. 


### For improving code's scaling performance,
### 1. Used rand_r for random number generation. </br> 2. Used the number of cores for seed generation. </br>3. Implemented reduction variable with + operator 

![Problem3_PartBImage.png](Problem3_PartBImage.png)

## Part C

### N= ${4^{20}}$, using 128 cores, the integral value is 4.0108 which seems to be accurate as we got the same result multiple times by one digit accuracy.

![Problem3_PartCImage.png](Problem3_PartCImage.png)

## Bonus_Points
 
## Part A
#### Modified the code to output the value  $\hat{I}(4k) - \hat{I}(4k+1)$  along with the value of the integral $\hat{I}(4k)$


```c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <omp.h>

double function(double x[], int dim);
double sampleInterval(double, double);

int main(int argc, char **argv) 
{
    double startTime = omp_get_wtime();
    unsigned long long N = atoll(argv[1]);
    
    double a = -1.0;
    double b = 1.0;

    const int dimension = atoll(argv[2]);
    double x[dimension];
    double V = 1024;
    double sum = 0.0;
    srand(time(NULL));
    unsigned long long i;
    int Num=pow(4,N+1);

    const int cores=atoll(argv[3]);
    omp_set_num_threads(cores);
    double sumi =0.0; 
    double absolute_error=0.0;
    double integral=0.0;
    double intg=0.0;

    for (i = 1; i <= Num; i++) 
    {
        for (int j = 0; j < dimension; j++) 
        {
            x[j] = sampleInterval(a, b);
        }

        double func_value = function(x, dimension);

        sum = sum + func_value;
       
        for(int k=2;k<=N;k++)
        {
            if(i == pow(4,k))
            {
                integral = (V / (i)) * (sumi + sum);
                absolute_error=fabs(integral-intg);
                intg = integral;
                if(pow(4,k)-pow(4,k-1))
                printf("N=%llu  integral=%f absolute_error=%f volumn=%f\n", i, integral, absolute_error,V);
                sumi+=sum;
                sum = 0;
            }
        }
        
    }
    double endTime = omp_get_wtime();

    return 0;
}

double function(double x[], int dim) 
{
    double sum = 1.0;
    for (int i = 0; i < dim; i++) 
    {
        sum = sum + x[i];
    }
    return sum;
}

double sampleInterval(double a, double b) 
{
    double x = ((double)(rand() / (double)RAND_MAX) * (b - a)) + a;
    return x;
}
```

#### We passed k=4, and printed the integrals, $\hat{I}(4k)$, along with absolute error, $\hat{I}(4k) - \hat{I}(4k+1)$

![Bonus_PartAImage.png](Bonus_PartAImage.png)

## Part B

### plot of actual error, $\hat{I}(4k) - 2^{10}$, the estimated error, and the theoretically expected error 

![Bonus_PartBImage.png](Bonus_PartBImage.png)

## Part C

### Parallelized the program using OpenMP reduction. To make one thread print the integral, we restructured the code and used omp critical, so that only one thread will have access to compute and print the integral.

```c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <omp.h>
 
double function(double x[], int dim);
double variable(double a, double b, unsigned int *seed);
 
int main(int argc, char **argv) {
    double startTime = omp_get_wtime();
    unsigned long long N = atoll(argv[1]);
 
    double a = -1.0;
    double b = 1.0;
   
    const int dimension = atoll(argv[2]);
    double x[dimension];
    double V = 1024;
    unsigned int seed = (unsigned int)time(NULL);
    unsigned long long i;
    int Num = pow(4, N);
    const int cores = atoll(argv[3]);
    omp_set_num_threads(cores);
    double sumi = 0.0;
    double integral = 0.0;
    double intg = 0.0;
 
    #pragma omp parallel private(x, seed, i) reduction(+ : sumi, integral, intg)
    {
        double sum = 0.0;
 
        #pragma omp for
        for (i = 1; i <= Num; i++) {
            for (int j = 0; j < dimension; j++) {
                x[j] = variable(a, b, &seed);
            }
 
            double func_value = function(x, dimension);
 
            sum += func_value;
 
            for (int k = 2; k <= N; k++) {
                if (i == pow(4, k)) {
                    #pragma omp critical
                    {
                        integral = (V / (i)) * (sumi + sum);
                        intg = integral;
                        printf("%llu %f %f\n", i, integral, V);
                        sumi += sum;
                        sum = 0.0; // Reset private sum for each thread
                    }
                }
            }
        }
    }
 
    double endTime = omp_get_wtime();
    return 0;
}
 
double function(double x[], int dim) {
    double sum = 1.0;
    for (int i = 0; i < dim; i++) {
        sum = sum + x[i];
    }
    return sum;
}
 
double variable(double a, double b, unsigned int *seed) {
    double x = ((double)(rand_r(seed) / (double)RAND_MAX) * (b - a)) + a;
    return x;
}
```

![bonus_PartCImage.png](bonus_PartCImage.png)